In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [13]:
import tensorflow as tf
# tensorFlow 코드를 보다 유연하게 작성가능 
# 계산 그래프를 구성하는 작업과 그 그래프를 실행하는 작업을 분리
sess = tf.InteractiveSession()

## 소프트맥스 회귀 모델 구성
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.global_variables_initializer())

y = tf.nn.softmax(tf.matmul(x, W) + b)

# tf.reduce_sum 은 모든 클래스에 대해 결과를 합하는 함수
# tf.reduce_mean 은 사용된 이미지들 각각에서 계산된 합의 평균을 구하는 함수
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices = [1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

for i in range(1000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(accuracy.eval(feed_dict={x: mnist.test.images, y_:mnist.test.labels}))

sess.close()

/Users/actmember/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


0.9171


In [18]:
import tensorflow as tf
# tensorFlow 코드를 보다 유연하게 작성가능 
# 계산 그래프를 구성하는 작업과 그 그래프를 실행하는 작업을 분리
sess = tf.InteractiveSession()

## 소프트맥스 회귀 모델 구성
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.global_variables_initializer())

y = tf.nn.softmax(tf.matmul(x, W) + b)

# 대칭성을 깨뜨리고 기울기가 0이 되는것을 방지하기 위해, 약간의 작업을 주어 초기화 
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev =0.1)
    return tf.Variable(initial)

# 죽은 뉴런을 방지하기 위해 편향을 작은 양수(0.1) 로 초기화
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# tensorflow는 합성곱과 풀링 계층(layer)을 유연하게 다룰 수 있도록 해줌
# padding 과 스트라이드(stride) 에 대한 선택
# 스트라이드 1, 출력 크기가 입력과 같게 되도록 0으로 패딩
def conv2d(x,W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

# 풀링은 2*2 크기의 맥스 풀링을 적용 
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

## 합성곱 계층 5*5의 윈도우(patch) 크기를 가지는 32개의 필터
# 처음 2개는 윈도우의 크기, 세번째는 입력 채널의 수, 마지막은 출력 채널 수
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# 이미지를 입력하기 위해 x를 4D 텐서로 reshape
x_image = tf.reshape(x, [-1, 28, 28, 1])

# relu 함수 적용, 출력값을 구하기 위해 마지막으로 맥스 풀링을 적용
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

## 두번째 합성곱 계층
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

## 완전 연결 계층(Fully-Connected Layer)
# 7 * 7 로 줄어든 이미지의 크기를 1024개의 뉴런으로 연결되는 완전 연결 계층을 구성
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# 드롭아웃
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 최종 소프트맥스 계층
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

# tf.reduce_sum 은 모든 클래스에 대해 결과를 합하는 함수
# tf.reduce_mean 은 사용된 이미지들 각각에서 계산된 합의 평균을 구하는 함수
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices = [1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_:batch[1], keep_prob:1.0
        })
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0
}))


/Users/actmember/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


step 0, training accuracy 0.1
step 100, training accuracy 0.84
step 200, training accuracy 0.86
step 300, training accuracy 1
step 400, training accuracy 0.92
step 500, training accuracy 0.92
step 600, training accuracy 0.96
step 700, training accuracy 0.96
step 800, training accuracy 0.96
step 900, training accuracy 0.88
step 1000, training accuracy 0.96
step 1100, training accuracy 0.96
step 1200, training accuracy 0.96
step 1300, training accuracy 0.98
step 1400, training accuracy 0.94
step 1500, training accuracy 0.98
step 1600, training accuracy 0.96
step 1700, training accuracy 0.98
step 1800, training accuracy 1
step 1900, training accuracy 0.92
step 2000, training accuracy 1
step 2100, training accuracy 1
step 2200, training accuracy 0.96
step 2300, training accuracy 0.92
step 2400, training accuracy 0.98
step 2500, training accuracy 0.94
step 2600, training accuracy 0.94
step 2700, training accuracy 1
step 2800, training accuracy 0.98
step 2900, training accuracy 1
step 3000, 